In [4]:
import pandas as pd
import matplotlib.pyplot as plt
pd.low_memory=False
import random

df = pd.read_csv("URL Classification.csv")
df = df.drop('1',axis=1)
df.columns=['url','category']
data = pd.DataFrame(columns=['url','category'])

In [5]:
categories = df.category.unique().tolist()
for i in categories:
    index_list = df[df.category == i ].index.tolist()
    x = random.sample(index_list,5000)
    data = pd.concat([df.loc[x],data],axis=0)

In [7]:
data.to_csv('sample_data.csv',index=False)

In [9]:
pd.read_csv('sample_data.csv')

,url,category
0,http://users.belgacom.net/hoshinkido.belgium,Sports
1,http://gobearcats.cstv.com/sports/w-baskbl/cin...,Sports
2,http://www.hongluck.org,Sports
3,http://aikithai.tripod.com/,Sports
4,http://www.decathlon2000.ee/eng/index.php,Sports
...,...,...
74995,http://www.sweet18teen.com/xxxhentai/,Adult
74996,http://www.zieberts.com/,Adult
74997,http://www.unprivacy.com/,Adult
74998,http://www.sex-xxx-transexuals.com/sites/porn/...,Adult


In [10]:
"this is shitting"

'this is shitting'